In [183]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [223]:
tf.__version__
%reload_ext google.cloud.bigquery
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [224]:
tf.__version__

'2.3.4'

In [185]:
import os

In [186]:
from google.cloud.bigquery import magics
from google.oauth2 import service_account
credentials = (service_account
    .Credentials.from_service_account_file(
        '/home/jupyter/handicap-tensorflow-323422-7540b32c20d3.json'))
magics.context.credentials = credentials

In [187]:
%%bigquery nba_measurements --verbose
SELECT * FROM `handicap-tensorflow-323422.nba_draft_combine_all_years.nba_draft_player_measurements` LIMIT 5

Executing query with job ID: 64a2f8ae-8a95-44b3-915f-07adfb2c57bb
Query executing: 0.57s


ERROR:
 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

(job ID: 64a2f8ae-8a95-44b3-915f-07adfb2c57bb)

                                         -----Query Job SQL Follows-----                                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT * FROM `handicap-tensorflow-323422.nba_draft_combine_all_years.nba_draft_player_measurements` LIMIT 5
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


In [188]:
QUERY = """
SELECT Draft_pick, Wingspan_minus_height, standing_reach_minus_height, Body_Fat FROM `handicap-tensorflow-323422.nba_draft_combine_all_years.nba_draft_player_measurements` """

In [189]:
client = bigquery.Client()

In [190]:
Query_Results = client.query(QUERY)


In [191]:
df = Query_Results.to_dataframe()

In [192]:
df.head()

,Draft_pick,Wingspan_minus_height,standing_reach_minus_height,Body_Fat
0,1.0,2.75,24.50,8.2
1,11.0,4.00,26.50,5.1
2,12.0,6.25,26.50,4.4
3,13.0,3.25,25.75,8.5
4,14.0,6.00,29.00,5.2


In [193]:
df.shape

(517, 4)

In [194]:
fdf = df.dropna()

In [195]:
fdf

,Draft_pick,Wingspan_minus_height,standing_reach_minus_height,Body_Fat
0,1.0,2.75,24.50,8.2
1,11.0,4.00,26.50,5.1
2,12.0,6.25,26.50,4.4
3,13.0,3.25,25.75,8.5
4,14.0,6.00,29.00,5.2
...,...,...,...,...
489,51.0,2.75,23.25,6.9
490,52.0,5.00,26.50,8.3
491,53.0,8.25,26.50,6.4
492,54.0,3.50,25.75,8.8


In [196]:
tf.keras.backend.clear_session()

In [197]:
train_df = fdf.sample(frac=0.8, random_state=123)
test_df = fdf.drop(train_df.index)

train_labels = train_df.pop('Draft_pick')
test_labels = test_df.pop('Draft_pick')

In [198]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation = 'linear', input_shape = [len(train_df.keys())])
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(), 
              loss='mean_squared_error',
              metrics=['mean_absolute_error', 'mean_squared_error'])

model.summary()

history = model.fit(train_df, train_labels,
                    epochs=10,
                    callbacks=[tf.keras.callbacks.TensorBoard("logs/multinominal-regression2")])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2021-08-22 02:01:19.098253: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


10/10 [==============================] - 0s 11ms/step - loss: 936.4736 - mean_absolute_error: 26.0711 - mean_squared_error: 936.4736
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 910.2130 - mean_absolute_error: 25.6014 - mean_squared_error: 910.2130
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 889.5907 - mean_absolute_error: 25.2422 - mean_squared_error: 889.5907
Epoch 4/10
 1/10 [==>...........................] - ETA: 0s - loss: 806.0983 - mean_absolute_error: 22.6570 - mean_squared_error: 806.0983

2021-08-22 02:01:19.641213: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2021-08-22 02:01:19.702921: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/multinominal-regression2/train/plugins/profile/2021_08_22_02_01_19
2021-08-22 02:01:19.706262: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/multinominal-regression2/train/plugins/profile/2021_08_22_02_01_19/tensorflow-2-3-20210820-044329.trace.json.gz
2021-08-22 02:01:19.711117: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/multinominal-regression2/train/plugins/profile/2021_08_22_02_01_19
2021-08-22 02:01:19.711284: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for memory_profile.json.gz to logs/multinominal-regression2/train/plugins/profile/2021_08_22_02_01_19/tensorflow-2-3-20210820-044329.memory_profile.json.gz
2021-08-22 02:

10/10 [==============================] - 0s 3ms/step - loss: 870.3757 - mean_absolute_error: 24.8990 - mean_squared_error: 870.3757
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 851.5245 - mean_absolute_error: 24.5643 - mean_squared_error: 851.5245
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 833.2455 - mean_absolute_error: 24.2348 - mean_squared_error: 833.2455
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 815.2751 - mean_absolute_error: 23.9066 - mean_squared_error: 815.2751
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 797.7580 - mean_absolute_error: 23.5890 - mean_squared_error: 797.7580
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 780.9105 - mean_absolute_error: 23.2754 - mean_squared_error: 780.9105
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 763.9594 - mean_absolute_error: 22.9766 - mean_squared_error: 763.9594


In [199]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mean_absolute_error,mean_squared_error,epoch
5,833.245544,24.234823,833.245544,5
6,815.275146,23.906603,815.275146,6
7,797.757996,23.589046,797.757996,7
8,780.910461,23.275446,780.910461,8
9,763.959351,22.976578,763.959351,9


In [200]:
test_predictions = model.predict(test_df).flatten()
results_df = pd.DataFrame({'Predicted': test_predictions, 'Actual': test_labels})
results_df.head(5)

,Predicted,Actual
1,9.000764,11.0
2,8.055774,12.0
3,7.022666,13.0
17,7.071517,26.0
18,8.572504,27.0


In [174]:
%load_ext google.datalab.kernel

In [175]:
from google.datalab.ml import TensorBoard as tb

In [178]:
tb.start('home/jupyter/logs/multinominal-regression1')

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:41251/ (Press CTRL+C to quit)


TensorBoard was started successfully with pid 3959. Click here to access it.

3959

In [201]:
%tensorboard --logdir='/home/jupyter/logs/multinomial-regression'


In [202]:
tf.saved_model.save(model, 'model/1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-08-22 03:02:44.663476: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/1/assets


In [208]:
%%bash
gsutil cp -r model/1/ gs://staging.handicap-tensorflow-323422.appspot.com/

Copying file://model/1/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://model/1/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://model/1/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [3 files][ 48.7 KiB/ 48.7 KiB]                                                
Operation completed over 3 objects/48.7 KiB.                                     


In [212]:
%%bash
gcloud ai-platform models create nba_measurement_multinominal_regression

Using endpoint [https://us-central1-ml.googleapis.com/]
API [ml.googleapis.com] not enabled on project [531848588786]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  
ERROR: (gcloud.ai-platform.models.create) User [riveryth@outlook.com] does not have permission to access projects instance [handicap-tensorflow-323422] (or it may not exist): AI Platform Training & Prediction API has not been used in project 531848588786 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=531848588786 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=531848588786
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googlea

CalledProcessError: Command 'b'gcloud ai-platform models create nba_measurement_multinominal_regression\n'' returned non-zero exit status 1.

In [222]:
%%bash
MODEL_DIR="gs://staging.handicap-tensorflow-323422.appspot.com/1"
VERSION_NAME="v1"
MODEL_NAME="nba_measurement_mutinominal_regression"
FRAMEWORK="tensorflow"

gcloud ai-platform versions create $VERSION_NAME \
    --model=$MODEL_NAME \ 
    --origin=$MODEL_DIR \
    --runtime-version=1.14 \
    --framework=%FRAMEWORK \
    --python-version=3.9

ERROR: (gcloud.ai-platform.versions.create) unrecognized arguments:   

To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS
bash: line 9: --origin=gs://staging.handicap-tensorflow-323422.appspot.com/1: No such file or directory


CalledProcessError: Command 'b'MODEL_DIR="gs://staging.handicap-tensorflow-323422.appspot.com/1"\nVERSION_NAME="v1"\nMODEL_NAME="nba_measurement_mutinominal_regression"\nFRAMEWORK="tensorflow"\n\ngcloud ai-platform versions create $VERSION_NAME \\\n    --model=$MODEL_NAME \\ \n    --origin=$MODEL_DIR \\\n    --runtime-version=1.14 \\\n    --framework=%FRAMEWORK \\\n    --python-version=3.9\n'' returned non-zero exit status 127.